In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
import random
from scipy.stats import norm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from copy import copy

pd.set_option('display.max_rows', 500)

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS'])))
query = """
    select * from research.fish_metadata a left join keypoint_annotations b
    on a.left_url = b.left_image_url 
    where b.keypoints is not null
    and (b.is_qa = false or b.captured_at > '2019-09-19');
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
blacklisted_keypoint_annotation_ids = [
    606484, 
    635806, 
    637801, 
    508773, 
    640493, 
    639409, 
    648536, 
    507003,
    706002,
    507000,
    709298,
    714073,
    719239
]

df = df[~df.id.isin(blacklisted_keypoint_annotation_ids)]

<h1> Apend world keypoints to this data </h1>

In [ ]:
BODY_PARTS = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE',
    'UPPER_PRECAUDAL_PIT', 
    'LOWER_PRECAUDAL_PIT',
    'HYPURAL_PLATE'
])


In [ ]:
def well_behaved(wkps, cutoff_depth=10.0):
    if any([abs(wkp[1]) > cutoff_depth for wkp in wkps.values()]):
        return False
    return True
    

def flip_center_kps(left_kps, right_kps):
    
    x_min_l = min([kp[0] for kp in left_kps.values()])
    x_max_l = max([kp[0] for kp in left_kps.values()])
    x_mid_l = np.mean([x_min_l, x_max_l])
    
    y_min_l = min([kp[1] for kp in left_kps.values()])
    y_max_l = max([kp[1] for kp in left_kps.values()])
    y_mid_l = np.mean([y_min_l, y_max_l])
    
    x_min_r = min([kp[0] for kp in right_kps.values()])
    x_max_r = max([kp[0] for kp in right_kps.values()])
    x_mid_r = np.mean([x_min_r, x_max_r])
    
    y_min_r = min([kp[1] for kp in right_kps.values()])
    y_max_r = max([kp[1] for kp in right_kps.values()])
    y_mid_r = np.mean([y_min_r, y_max_r])
        
    fc_left_kps, fc_right_kps = {}, {}
    flip_factor = 1 if left_kps['UPPER_LIP'][0] > left_kps['TAIL_NOTCH'][0] else -1
    for bp in BODY_PARTS:
        left_kp, right_kp = left_kps[bp], right_kps[bp]
        if flip_factor > 0:
            fc_left_kp = np.array([left_kp[0] - x_mid_l, left_kp[1] - y_mid_l])
            fc_right_kp = np.array([right_kp[0] - x_mid_l, right_kp[1] - y_mid_l])
        else:
            fc_right_kp = np.array([x_mid_r - left_kp[0], left_kp[1] - y_mid_r])
            fc_left_kp = np.array([x_mid_r - right_kp[0], right_kp[1] - y_mid_r])
        fc_left_kps[bp] = fc_left_kp
        fc_right_kps[bp] = fc_right_kp
        
    return fc_left_kps, fc_right_kps


def _rotate_cc(p, theta):
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])
    
    rotated_kp = np.dot(R, p)
    return rotated_kp


def rotate_kps(left_kps, right_kps):
    upper_lip_x, upper_lip_y = left_kps['UPPER_LIP']
    theta = np.arctan(upper_lip_y / upper_lip_x)
    r_left_kps, r_right_kps = {}, {}
    for bp in BODY_PARTS:
        rotated_kp = _rotate_cc(left_kps[bp], -theta)
        r_left_kps[bp] = rotated_kp
        disp = abs(left_kps[bp][0] - right_kps[bp][0])
        r_right_kps[bp] = np.array([rotated_kp[0] - disp, rotated_kp[1]])
        
    return r_left_kps, r_right_kps


def translate_kps(left_kps, right_kps, factor):
    t_left_kps, t_right_kps = {}, {}
    for bp in BODY_PARTS:
        left_kp, right_kp = left_kps[bp], right_kps[bp]
        t_left_kps[bp] = factor * np.array(left_kps[bp])
        t_right_kps[bp] = factor * np.array(right_kps[bp])
    
    return t_left_kps, t_right_kps


def jitter_kps(left_kps, right_kps, jitter):
    j_left_kps, j_right_kps = {}, {}
    for bp in BODY_PARTS:
        j_left_kps[bp] = np.array([left_kps[bp][0] + np.random.normal(0, jitter), 
                                   left_kps[bp][1] + np.random.normal(0, jitter)])
        j_right_kps[bp] = np.array([right_kps[bp][0] + np.random.normal(0, jitter), 
                                    right_kps[bp][1] + np.random.normal(0, jitter)])
    
    return j_left_kps, j_right_kps



def modify_kps(left_kps, right_kps, factor, jitter, cm):
    fc_left_kps, fc_right_kps = flip_center_kps(left_kps, right_kps)
    r_left_kps, r_right_kps = rotate_kps(fc_left_kps, fc_right_kps)
    t_left_kps, t_right_kps = translate_kps(r_left_kps, r_right_kps, factor)
    j_left_kps, j_right_kps  = jitter_kps(t_left_kps, t_right_kps, jitter)
    j_left_kps_list, j_right_kps_list = [], []
    for bp in BODY_PARTS:
        l_item = {
            'keypointType': bp,
            'xFrame': j_left_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
            'yFrame': j_left_kps[bp][1] + cm['pixelCountHeight'] / 2.0
        }
        
        r_item = {
            'keypointType': bp,
            'xFrame': j_right_kps[bp][0] + cm['pixelCountWidth'] / 2.0,
            'yFrame': j_right_kps[bp][1] + cm['pixelCountHeight'] / 2.0
        }
        
        j_left_kps_list.append(l_item)
        j_right_kps_list.append(r_item)
        
    modified_kps = {
        'leftCrop': j_left_kps_list,
        'rightCrop': j_right_kps_list
    }
    
    return modified_kps


def process_df(df, n_factors=1, jitters=[0, 10, 20], low=0.3, high=2, oos=False, network=None):
    features_data = defaultdict(list)
    
    row_count = 0
    for idx, row in df.iterrows():
        keypoints = row.keypoints
        left_keypoints_list = keypoints.get('leftCrop')
        right_keypoints_list = keypoints.get('rightCrop')
        cm = row.camera_metadata
        if left_keypoints_list and right_keypoints_list:
            wkps = pixel2world(left_keypoints_list, right_keypoints_list, cm)
            left_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in left_keypoints_list}
            right_kps = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in right_keypoints_list}
            if well_behaved(wkps):
                for n in range(n_factors):
                    factor = 1.0 if n_factors == 1 else np.random.uniform(low=low, high=high)
                    for jitter in jitters:
                        trials = 3 if jitter > 0 else 1
                        for t in range(trials):
                            modified_kps = modify_kps(left_kps, right_kps, factor, jitter, cm)
                            modified_wkps = pixel2world(modified_kps['leftCrop'], modified_kps['rightCrop'], cm)

                            for i in range(len(BODY_PARTS)-1):
                                for j in range(i+1, len(BODY_PARTS)):
                                    d = euclidean_distance(modified_wkps[BODY_PARTS[i]], 
                                                           modified_wkps[BODY_PARTS[j]])
                                    features_data['{0}-{1}'.format(i, j)].append(d)
                            
                            features_data['weight'].append(row.weight)
                            features_data['captured_at'].append(row.captured_at)
                            features_data['gtsf_fish_identifier'].append(row.fish_id)
                            features_data['keypoint_annotation_id'].append(row.id)
                            features_data['world_keypoints'].append(modified_wkps)
        if row_count % 1000 == 0:
            print('Percentage complete: {}'.format(row_count / df.shape[0]))
        row_count += 1
                            
    
    return features_data






In [ ]:
features_data = process_df(df)

In [ ]:
features_df = pd.DataFrame(features_data)

In [ ]:
# define all features

body_parts_subset = sorted([
    'HYPURAL_PLATE',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
])

body_part_indices = [BODY_PARTS.index(bp) for bp in body_parts_subset]

pairwise_distance_columns = ['{0}-{1}'.format(x, y) for x, y in list(combinations(body_part_indices, 2))]
interaction_columns_quadratic = []
interaction_columns_cubic = []
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        col1 = pairwise_distance_columns[i]
        col2 = pairwise_distance_columns[j]
        interaction_column = '{},{}'.format(col1, col2)
        features_df[interaction_column] = features_df[col1] * features_df[col2]
        interaction_columns_quadratic.append(interaction_column)
        
for i in range(len(pairwise_distance_columns)):
    for j in range(i, len(pairwise_distance_columns)):
        for k in range(j, len(pairwise_distance_columns)):
            col1 = pairwise_distance_columns[i]
            col2 = pairwise_distance_columns[j]
            col3 = pairwise_distance_columns[k]
            interaction_column = '{},{},{}'.format(col1, col2, col3)
            features_df[interaction_column] = features_df[col1] * features_df[col2] * features_df[col3]
            interaction_columns_cubic.append(interaction_column)
            

<h1> Train Linear Model </h1>

In [ ]:
weights = []
i = 0
for idx, row in features_df.iterrows():
    if i % 100 == 0:
        print(i)
    i += 1
    count = features_df[features_df.gtsf_fish_identifier == row.gtsf_fish_identifier].shape[0]
    if count > 1:
        weights.append(1.0 / count ** 0.5)
#         weights.append(1.0 / count)
    else:
        weights.append(1)
        
features_df['w'] = weights

In [ ]:
def generate_train_mask(df, train_frac, randomize=True):
    x = np.zeros((df.shape[0]), dtype=bool)
    x[:int(train_frac * df.shape[0])] = True
    np.random.shuffle(x)
    mask = pd.Series(x)
    return x
    

def generate_oos_score(features_df, mask, train_size, num_eigenvectors):
    np.random.seed(0)
    columns = pairwise_distance_columns + interaction_columns_quadratic + interaction_columns_cubic

    X_train = features_df.loc[mask, columns].values
    y_train = features_df.loc[mask, 'weight'].values
    w_train = features_df.loc[mask, 'w'].values
    X_test = features_df.loc[~mask, columns].values
    y_test = features_df.loc[~mask, 'weight'].values

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_normalized = scaler.transform(X_train)

    pca = PCA(n_components=min(X_train_normalized.shape[0], X_train_normalized.shape[1]))
    pca.fit(X_train_normalized)
    explained_variance_ratio = pca.explained_variance_ratio_.cumsum()
    idx = num_eigenvectors

    pca = PCA(n_components=idx+1)
    pca.fit(X_train_normalized)
    X_train_transformed = pca.transform(X_train_normalized)
    X_test_normalized = scaler.transform(X_test)
    X_test_transformed = pca.transform(X_test_normalized)

    reg = LinearRegression().fit(X_train_transformed, y_train, sample_weight=w_train)
    score = reg.score(X_test_transformed, y_test)

    y_pred = reg.predict(pca.transform(scaler.transform(features_df[columns].values)))
    features_df['prediction'] = y_pred
    features_df['error'] = features_df.prediction - features_df.weight
    features_df['error_pct'] = features_df.error / features_df.weight
    features_df['abs_error_pct'] = features_df.error_pct.abs()

    model = {
    'mean': scaler.mean_,
    'std': scaler.scale_,
    'PCA_components': pca.components_,
    'reg_coef': reg.coef_,
    'reg_intercept': reg.intercept_,
    'body_parts': body_parts_subset   
    }
    

    return mask, model, score



In [ ]:
# num eigenvectors = 20

train_size = 2000
gtsf_fish_identifiers = list(features_df.gtsf_fish_identifier.unique())
fish_ids = random.sample(gtsf_fish_identifiers, train_size)
date_mask = (features_df.captured_at < '2019-09-01') 
mask = date_mask & features_df.gtsf_fish_identifier.isin(fish_ids)
mask, model, score = generate_oos_score(features_df, mask, 2000, 20)

In [ ]:
%matplotlib inline
amg = AccuracyMetricsGenerator()
test_mask = date_mask & ~features_df.gtsf_fish_identifier.isin(fish_ids)
amg.set_data(mask, features_df.prediction.values, features_df.weight.values, w=features_df.w.values, test_mask=test_mask)
amg.plot_predictions_vs_ground_truth(impose_bounds=True)

In [ ]:
amg.display_train_test_accuracy_metrics()

In [ ]:
%matplotlib inline
amg = AccuracyMetricsGenerator()
test_mask = ~date_mask
amg.set_data(mask, features_df.prediction.values, features_df.weight.values, w=features_df.w.values, test_mask=test_mask)
amg.plot_predictions_vs_ground_truth(impose_bounds=False)

In [ ]:
amg.display_train_test_accuracy_metrics()

In [ ]:
%matplotlib inline
mask = features_df.captured_at < '2019-09-10'
fig, ax = plt.subplots(figsize=(20, 10))
sc = ax.scatter(features_df[~mask].weight.values, features_df[~mask].prediction.values, c=features_df[~mask].depth.values)
plt.colorbar(sc)
plt.plot([0, 5e3], [0, 5e3], color='red')
plt.xlim([0, 5e3])
plt.ylim([0, 2e4])
plt.xlabel('Ground truth weight (grams)')
plt.ylabel('Predicted weight (grams)')
plt.title('Prediction vs. Ground Truth: fish far away from camera, Linear PCA')
plt.grid()
plt.show()